# analyze_reaction_times

Performs analysis of reaction time data for subjects categorizing images of animals. The data is simulated and the code is meant only to demonstrate basics of paths and file handling. See the `README.md` for further information.

Part of the *Computational Fluency Short Course* offered in 2023 by Brown University's Carney Institute. Intended only for educational purposes; use at your own risk.

In [1]:
# Import block
# Loads packages that extends base python

import os  # Operating system utilities

import pandas as pd  # Common package for working with tabular data

## Getting paths to the data

As described in the course materials, different operating systems use slightly different conventions for describing paths. The most notable differences are that Linux and Mac use `/` as a separator while Windows uses `\`, and the root directory on Linux and Max is `/`, while the Windows root is (usually) `C:\`.

In order to write code that works across operating systems, we have to check the OS and figure out the right convention to use. As a convenience, `python` provides the `os` module that does the work for us.

A second issue is that we have to know where the data is, but if we use an absolute path that is correct on our computer, it will break when moved to a different computer. One way we can try to handle this situation is with *relative paths*, assuming we know the working directory set when the notebook starts up (sometimes this breaks also, especially on remote systems, and some troubleshooting is required). By default, opening a notebook within Jupyter Lab sets that notebook's local as the working dirtectory. 

In [2]:
# Put together a path to the data

# Look at the python kernel's working directory
# Note this directory may not match wherever you started Jupyter Lab
cwd = os.getcwd()  
print(f'Current working directory:\n{cwd}\n')

# Make a relative path to the data
data_dir_rel = os.path.join('..','testdata')  # join adds the correct path separator
# Doube dots ".." mean "go up one directory"
print(f'Relative path to data:\n{data_dir_rel}\n')

# Convert a relative path to absolute path
data_dir = os.path.abspath(data_dir_rel)  
print(f'Absolute path to data:\n{data_dir}\n')

Current working directory:
/Users/lewis26nz/Documents/GitHub/cfsc2023-paths/notebooks

Relative path to data:
../testdata

Absolute path to data:
/Users/lewis26nz/Documents/GitHub/cfsc2023-paths/testdata



Now we look in the data directory to find subject-wise subdirectories, and continue down to the data files.

**Note**: It's usually a bad idea to blindly loop through everything within a directory. There will often be other kinds of files in there, either intentionally (like an index of subjects) or unintenitionally (automatically generated files and/or dot-files).

In [3]:
print(f'Content of {data_dir}:\n')
for name in os.listdir(data_dir):
    is_subject = str.startswith(name,'sub-')  # Test the name
    if is_subject:
        print(f'Found subject data directory: {name}')
    else:
        print(f'Skipping other file {name}')
print('\n')

Content of /Users/lewis26nz/Documents/GitHub/cfsc2023-paths/testdata:

Skipping other file participants.csv
Found subject data directory: sub-1
Found subject data directory: sub-2
Found subject data directory: sub-3




## Manually load some data

As a demonstration, we load a single data file, and look at its contents.

In [4]:
filepath = os.path.join(data_dir,'sub-1','sub-1_task-class_beh.csv')
print(f'Loading data at {filepath}')

df = pd.read_csv(filepath)  # Load data as a Pandas DataFrame

Loading data at /Users/lewis26nz/Documents/GitHub/cfsc2023-paths/testdata/sub-1/sub-1_task-class_beh.csv


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SubjectID     15 non-null     int64  
 1   TrialNum      15 non-null     int64  
 2   StimulusID    15 non-null     object 
 3   TrialType     15 non-null     object 
 4   Response      15 non-null     object 
 5   ReactionTime  15 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 848.0+ bytes


In [6]:
df

,SubjectID,TrialNum,StimulusID,TrialType,Response,ReactionTime
0,1,1,d05,dog,cat,0.5555
1,1,2,b04,bird,bird,0.8708
2,1,3,b01,bird,bird,0.8358
3,1,4,c02,cat,cat,1.1063
4,1,5,d03,dog,dog,0.7613
5,1,6,b05,bird,bird,1.1252
6,1,7,b03,bird,bird,1.0052
7,1,8,c05,cat,cat,0.8584
8,1,9,c04,cat,dog,0.6823
9,1,10,b02,bird,bird,0.8660


In [7]:
# Extract columns by name
print('The set of reaction times is\n', df["ReactionTime"] )

# Extract operations on columns via "methods"
print(f'\nThe mean reaction time is { df["ReactionTime"].mean() } seconds')

The set of reaction times is
 0     0.5555
1     0.8708
2     0.8358
3     1.1063
4     0.7613
5     1.1252
6     1.0052
7     0.8584
8     0.6823
9     0.8660
10    0.3949
11    0.8764
12    0.8045
13    0.7177
14    0.8721
Name: ReactionTime, dtype: float64

The mean reaction time is 0.82216 seconds


In [10]:
# Optional Exercise: find the standard deviation of RTs
import numpy as np
sd = np.std(df["ReactionTime"])
sd = df["ReactionTime"].std()

In [13]:
# Optional Exercise: use the above results to make "z-scored" RTs, that is, 
#   subtract the mean and then divide by the standard deviation for all trials
reaction_z = (df["ReactionTime"] - df["ReactionTime"].mean()) / sd

In [14]:
# Optional Exercise: find the three mean RTs separately for each type of trial (this
# one is a little more advanced)
bird_mean = df[df["TrialType"] == "bird"] ["ReactionTime"] . mean()
dog_mean = df[df["TrialType"] == "dog"] ["ReactionTime"] . mean()
cat_mean = df[df["TrialType"] == "cat"] ["ReactionTime"] . mean()
print(f'Bird mean: {bird_mean}')
print(f'Dog mean: {dog_mean}')
print(f'Cat mean: {cat_mean}')

Bird mean: 0.9405999999999999
Dog mean: 0.69204
Cat mean: 0.83384


In [17]:
# Optional Exercise: find the two mean RTs separately for correct and incorrect
# trials (this one is harder still, but follows from the previous exercise)
correct_mean = df[df["TrialType"] == df ["Response"]]["ReactionTime"].mean()
incorrect_mean = df[df["TrialType"] != df ["Response"]]["ReactionTime"].mean() 
print (f'Correct mean: {correct_mean}')
print (f'Incorrect mean: {incorrect_mean}')

Correct mean: 0.8916416666666667
Incorrect mean: 0.5442333333333333


## Paths for loading code

Sometimes we'd prefer to use our own custom code across multiple notebooks, for example to do some standardized data loading or pre-processing. We then put that code into a module (bassically just a `.py` file) or package, and then `import` it in the notebook.

However, the python interpreter needs to know where to look for that code. It searches a "path list", that is automatically set when python is launched. 

In [ ]:
import sys  # Access python system properties

In [ ]:
path_list = sys.path

print('Python will search for code in the following directories (in order):\n')
for path in path_list:
    print(path)

Note the empty line (it is not missing, it is actually an empty string). Python interprets the empty string as meaning whatever the current working directory is.

We thus have several choices to load our code:
- Put the module in the same directory as the notebook
- Change the working directory to wherever the module is
- Change the system path to include the location of the module
- Package the module in a way that can be placed in a "global" location

There are many opinions about best practice, and it often depends on the details of your project (for example, how globally useful the extra code is, and how many people are sharing it). For code that will be reused often, making a package may be the best option, though it takes a little extra work.

We will demonstrate a kind of hack that can be useful in simple situations: we change directories to where the code should be, import it, then change back to where we started.

In [ ]:
# First try to import with just the module name (will fail !)

import datahandling as dh

In [ ]:
# Hack to import code in another directory without changing the path list

wd_orig = os.getcwd()

wd_code = os.path.abspath( os.path.join('..','utils') )
print(f'Attempt to load module in:\n{wd_code}\n')

# We put in a try-except block so even if there's an error during import 
# we still get to code that will restore the working directory
try:
    os.chdir(wd_code)
    import datahandling as dh
    print('Import succeeded')
except:  # If the try block crashes we end up here 
    print('Failed to load module; attempting to restore working directory ')

os.chdir(wd_orig)

In [ ]:
print(f'Current working directory is {os.getcwd()}')

Now let's use the utilities in `datahandling` to load the data.

In [ ]:
df2 = dh.load_subject(filepath)
df2.info()

In [ ]:
# Check that error handling in dh works as expected
df3 = dh.load_subject('/nonsense/file/path')
print(df3)